## Alex Felipe Barbosa - 07/07/2025
### Data Science Academy 
### Projetos de Analise de Dados com Linguagem Python

### Projeto 1 - Extraindo e Analisando Dados de Banco de Dados

Pacotes Python utilizados no Projeto

In [24]:
# Instala o pacote watermaark
!pip install -q -U watermark

In [25]:
# Instala o pacote iPython-sql
!pip install -q ipython-sql

In [26]:
# Imports 
import os
import pandas as pd
import sqlite3

In [27]:
%reload_ext watermark
%watermark -a "Alex Felipe Barbosa"

Author: Alex Felipe Barbosa



Criando o banco de dados relacional

In [28]:
# Cria um dataframe com os dados de origem
dados = pd.DataFrame({'nivel': ['Junior', 'Pleno','Senior'],
                      'salario': [7500,14650,18320],
                      'cargo': ['Analista de Dados', 'Cientista de Dados', 'Engenheiro de Dados']})

In [29]:
#Lista os dados
dados.head()

,nivel,salario,cargo
0,Junior,7500,Analista de Dados
1,Pleno,14650,Cientista de Dados
2,Senior,18320,Engenheiro de Dados


In [30]:
# Define o caminho para o arquivo do Banco de Dados
arquivo_path = 'database.db'

In [31]:
# Verifica se o arquivo existe e deleta se existir para criar um arquivo novo depois
if os.path.exists(arquivo_path):
  try:
    os.remove(arquivo_path)
    print(f"Arquivo {arquivo_path} deletado com sucesso!")
  except Exception as e:
    print(f"Erro ao deletar o arquivo {arquivo_path}. Detalhes: {e}")  
else:
  print(f"Arquivo {arquivo_path} não encontrado.")

Erro ao deletar o arquivo database.db. Detalhes: [WinError 32] O arquivo já está sendo usado por outro processo: 'database.db'


Conectando ao Banco de Dados com Linguagem Python

In [32]:
# Criamos a conexao a um banco de dados SQLite
cnn = sqlite3.connect('database.db')

In [33]:
# Copia o dataframe para dentro do banco de dados como uma tabela
dados.to_sql ('funcionarios', cnn)

ValueError: Table 'funcionarios' already exists.

In [ ]:
# Carregamos a extensão SQL
%load_ext sql

In [ ]:
# Definimos o banco de dados
%sql sqlite:///database.db

Agora executamos nossas consultas SQL usando diretamente Linguagem SQL dentro do Jupyter Notebook.

In [ ]:
%%sql

Select * from funcionarios

 * sqlite:///database.db
Done.


index,nivel,salario,cargo
0,Junior,7500,Analista de Dados
1,Pleno,14650,Cientista de Dados
2,Senior,18320,Engenheiro de Dados


In [ ]:
%%sql
select count(*) from funcionarios;

 * sqlite:///database.db
Done.


count(*)
3


Carregando o Banco de Dados a partir de Arquivos CSV


Temos em mãos um arquivo com dados de pacientes que desenvolveram ou não diabetes. Vamos colocar o conteúdo do arquivo em um Banco de Dados.

In [34]:
# Carrega o dataset
df = pd.read_csv('dataset.csv')

In [35]:
type(df)

pandas.core.frame.DataFrame

In [36]:
df.shape

(768, 9)

In [37]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,103,30,38,83,43.3,0.183,33,0
1,1,115,70,30,96,34.6,0.529,32,1
2,3,126,88,41,235,39.3,0.704,27,0
3,8,99,84,0,0,35.4,0.388,50,0
4,7,196,90,0,0,39.8,0.451,41,1


In [38]:
# Copia o dataframe para dentro do banco de dados como uma tabela
df.to_sql('diabetes', cnn)

768

In [43]:
%%sql 
Select count(*) from diabetes;

 * sqlite:///database.db
Done.


count(*)
768


Sintaxe SQL e Sintaxe Pandas

Retorne os pacientes que tem BMI maior que 52 e idade (Age) entre 25 e 30 anos.

In [40]:
# Sintaxe da consulta do Pandas:
df.query("BMI > 52 and 25 <= Age <= 30")

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
120,0,162,76,56,100,53.2,0.759,25,1
125,1,88,30,42,99,55.0,0.496,26,1
177,0,129,110,46,130,67.1,0.319,26,1
303,5,115,98,0,0,52.9,0.209,28,1
445,0,180,78,63,14,59.4,2.420,25,1


In [46]:
%%sql 
Select * from diabetes where BMI > 52 and age between 25 and 30

 * sqlite:///database.db
Done.


index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
120,0,162,76,56,100,53.2,0.759,25,1
125,1,88,30,42,99,55.0,0.496,26,1
177,0,129,110,46,130,67.1,0.319,26,1
303,5,115,98,0,0,52.9,0.209,28,1
445,0,180,78,63,14,59.4,2.42,25,1


# Respondendo perguntas de negócio com Análise de Dados
Precisamos gerar uma amostra de dados com pacientes com mais de 50 anos e para cada um deles indicar uma nova coluna se o paciente esta normal (BMI menor que 30) ou Obeso (BMI maior ou igual a 30). Então devemos gerar um novo CSV e encaminhar para o tomador de decisão.

Retorne Age, Glucose, OutCome para pacientes com Glucose maior que 195

In [47]:
%%sql
Select Age, Glucose, Outcome from diabetes where glucose > 195;

 * sqlite:///database.db
Done.


Age,Glucose,Outcome
41,196,1
53,197,1
57,196,1
31,197,0
29,196,1
39,197,1
28,198,1
62,197,1
22,199,1


Vamos criar uma nova tabela no banco de dados

In [48]:
%%sql 

CREATE TABLE pacientes (Pregnancies INT,
                        Glucose INT,
                        BloodPressure INT,
                        SkinThickness INT,
                        Insulin INT,
                        BMI DECIMAL(8,2),
                        DiabetesPedigreeFunction DECIMAL(8,2),
                        Age INT,
                        Outcome INT
                        );

 * sqlite:///database.db
Done.


[]

In [49]:
%%sql 

Select * from pacientes;

 * sqlite:///database.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


Vamos copiar o conteúdo de uma tabela para outra, mas somente para pacientes maiores que 50 anos de idade.

In [51]:
%%sql

INSERT INTO pacientes (Pregnancies,
                        Glucose,
                        BloodPressure,
                        SkinThickness,
                        Insulin,
                        BMI,
                        DiabetesPedigreeFunction,
                        Age,
                        Outcome)
SELECT Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome
FROM diabetes WHERE Age > 50;

 * sqlite:///database.db
81 rows affected.


[]

In [52]:
%%sql

Select * from pacientes;

 * sqlite:///database.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
2,197,70,45,543,30.5,0.158,53,1
8,125,96,0,0,0,0.232,54,1
10,139,80,0,0,27.1,1.441,57,0
1,189,60,23,846,30.1,0.398,59,1
5,166,72,19,175,25.8,0.587,51,1
11,143,94,33,146,36.6,0.254,51,1
13,145,82,19,110,22.2,0.245,57,0
5,109,75,26,0,36,0.546,60,0
4,111,72,47,207,37.1,1.39,56,1
9,171,110,24,240,45.4,0.721,54,1


Vamos alterar a tabela e incluir uma nova coluna

In [53]:
%%sql 

ALTER TABLE pacientes 
ADD Perfil VARCHAR(10);

 * sqlite:///database.db
Done.


[]

Agora vamos atualizar a coluna com o valor "Normal" se o BMI for menor que 30

In [54]:
%%sql

UPDATE pacientes 
SET Perfil = 'Normal'
WHERE BMI < 30;

 * sqlite:///database.db
38 rows affected.


[]

Agora vamos atualizar a colhna com o valor "Obeso" se o BMI for maior ou igual a 30

In [55]:
%%sql 

UPDATE pacientes
SET Perfil = 'Obeso'
where BMI >= 30;

 * sqlite:///database.db
43 rows affected.


[]

In [56]:
%%sql
Select * from pacientes;

 * sqlite:///database.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Perfil
2,197,70,45,543,30.5,0.158,53,1,Obeso
8,125,96,0,0,0,0.232,54,1,Normal
10,139,80,0,0,27.1,1.441,57,0,Normal
1,189,60,23,846,30.1,0.398,59,1,Obeso
5,166,72,19,175,25.8,0.587,51,1,Normal
11,143,94,33,146,36.6,0.254,51,1,Obeso
13,145,82,19,110,22.2,0.245,57,0,Normal
5,109,75,26,0,36,0.546,60,0,Obeso
4,111,72,47,207,37.1,1.39,56,1,Obeso
9,171,110,24,240,45.4,0.721,54,1,Obeso


### Retornando os dados para o Pandas e salvando o CSV

In [57]:
# Query
dsa_query = cnn.execute("Select * from Pacientes")

In [58]:
dsa_query

In [59]:
# List Comprehension para retornar metadados da tabela (nomes de colunas)
cols = [coluna[0] for coluna in dsa_query.description]

In [60]:
cols

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome',
 'Perfil']

In [61]:
# Gera o Dataframe
resultado = pd.DataFrame.from_records(data = dsa_query.fetchall(), columns = cols)

In [62]:
# Shape
resultado.shape

(81, 10)

In [63]:
# Visualização
resultado.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Perfil
0,2,197,70,45,543,30.5,0.158,53,1,Obeso
1,8,125,96,0,0,0.0,0.232,54,1,Normal
2,10,139,80,0,0,27.1,1.441,57,0,Normal
3,1,189,60,23,846,30.1,0.398,59,1,Obeso
4,5,166,72,19,175,25.8,0.587,51,1,Normal


In [64]:
# Salva em CSV
resultado.to_csv('resultado.csv', index = False)

In [65]:
%reload_ext watermark
%watermark -a "Alex Felipe Barbosa"

Author: Alex Felipe Barbosa

